Terminal: 

pip install -e C:\Users\ID\.bitbucket\cea\Wrapd 


In [2]:
!pip install -e C:\Users\ID\.bitbucket\cea\Wrapd 

import CEA_Wrapd as cea

Obtaining file:///C:/Users/ID/.bitbucket/cea/Wrapd
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for CEA-Wrapd (pyproject.toml): started
  Building editable for CEA-Wrapd (pyproject.toml): finished with status 'done'
  Created wheel for CEA-Wrapd: filename=CEA_Wrapd-1.6.1-0.editable-py3-none-any.whl size=21860 sha256=efe4673a4d4368b1907c57c16721e71ba2d474319ee52859e6df2df36b33bf57
  Stored in directory: C:\Users\ID\AppDa

https://pypi.org/project/CEA-Wrap/ 

Desired Reactants:
> C7H16,n-heptane
> C8H18,isooctane
> Air

In [7]:
import CEA_Wrapd as cea
import numpy as np 
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import OttoCycle as oc
import copy
import csv

In [4]:
hept = 'C7H16,n-heptane'
oct = 'C8H18,isooctane'
a = "Air"
f1 = cea.Fuel(hept, temp = 350, wt_percent = 13.00)
f2 = cea.Fuel(oct, temp = 350, wt_percent = 87.00)
o1 = cea.Oxidizer(a, temp = 320, wt_percent = 100)

In [12]:

m = [f1,f2,o1]
m87 = m
p1 = cea.UVProblem(
    filename = "first", 
    materials = m,
    density = 1.1731,
    density_units = "kg",
    massf = True
)
p1.set_o_f(12.0)

In [6]:
data = p1.run()
data
#data.cp

{'prod_c': {'Ar': 0.01192,
  'CO': 0.07734,
  'CO2': 0.11595,
  'H': 7e-05,
  'H2': 0.00167,
  'H2O': 0.09272,
  'NO': 0.00126,
  'N2': 0.6965,
  'O': 0.0001,
  'OH': 0.00198,
  'O2': 0.00048},
 'dLV_dLP_t': -1.00207,
 'dLV_dLT_p': 1.053,
 'visc': 8.933400000000001e-05,
 'p': 9.3745,
 't': 2612.7,
 'rho': 1.1731,
 'h': 585.72,
 'u': -213.41,
 'g': -24174.0,
 's': 9.4766,
 'm': 0.0,
 'mw': 27.184,
 'cp': 1.9335,
 'gammas': 1.2096,
 'phi': 1.2615,
 'son': 983.19,
 'cond': 3.2465,
 'pran': 0.53205,
 'gamma': 1.2121038720000001}

In [30]:

IGC_air = .287 # kJ/kgK
Qcomb = 45000 #kJ/kg
comb_eff = .9

Amb_Pres = 101 #kPa
Amb_Temp = 300 #K
Amb_Dens = 1.1731 #kg/m3
gamma_air = 1.4
comp_eff = .85
exp_eff = .9

af_deci = 12
disp_deci = 1660 * 1E-6
CylCt_deci = 4
CR_deci = 10
Cp = 1.005
Cv = Cp/gamma_air

cylvol = disp_deci / CylCt_deci
vol_deci = cylvol/CR_deci
Otto_deci= oc.Otto(Amb_Pres, Amb_Temp, Amb_Dens, vol_deci, CR_deci, gamma_air, comp_eff, exp_eff)
OtCyc_deci = oc.cyc_real(Otto_deci, "AFvsDISP", m87, True, af_deci, reac_warming= False, report_comb= True)

print(OtCyc_deci[5].get_density())

W = oc.cyc_work(OtCyc_deci, CR_deci, Cv, CylCt_deci, disp_deci)
print(W)

OtIdeal = oc.cyc_ideal(Otto_deci, IGC_air, Qcomb, Cp, comb_eff, af_deci)
W_ideal = oc.cyc_work(OtIdeal, CR_deci, Cv, CylCt_deci, disp_deci)
print(W_ideal)

#print(OtIdeal[0].curr_state())
#print(OtIdeal[1].curr_state())
#print(OtIdeal[2].curr_state())
#print(OtIdeal[3].curr_state())
#print(OtIdeal[4].curr_state())
#print(OtIdeal[5].curr_state())

Wcheck = (Cv * (OtIdeal[3].get_temperature() - OtIdeal[2].get_temperature()) - Cv * (OtIdeal[4].get_temperature() -OtIdeal[1].get_temperature())) #* (CylCt_deci * disp_deci)*(CR_deci/(CR_deci-1)) * OtIdeal[1].get_density()
Wcheck = (CylCt_deci * disp_deci)*(CR_deci/(CR_deci-1)) * OtIdeal[1].get_density()
print(Wcheck)


1.1731
5.556734079778884
10.792625222693953
0.000415
0.008654871111111111


In [ ]:

pv = OtCyc_deci[4].get_pressure() / OtCyc_deci[4].get_density() - OtCyc_deci[2].get_pressure() / OtCyc_deci[2].get_density()
W = oc.cyc_work(OtCyc_deci, CR, Cv, CylCt_deci, disp_deci)
Q = OtCyc_deci[6].h - pv - W
print(Q)

In [12]:
test = cea.HPProblem(materials = m, p_f = 14)
test.run()



{'prod_c': {'Ar': 0.00852,
  'CO': 0.02557,
  'CO2': 0.1065,
  'H': 0.00011,
  'H2': 0.00597,
  'H2O': 0.14228,
  'NO': 0.00026,
  'N2': 0.71023,
  'O': 1e-05,
  'OH': 0.00049,
  'O2': 5e-05},
 'dLV_dLP_t': -1.00027,
 'dLV_dLT_p': 1.0083,
 'visc': 8.2597e-05,
 'p': 68.947,
 't': 2339.1,
 'h': -107.39,
 'mw': 28.233,
 'cp': 1.5462,
 'gammas': 1.2398,
 'phi': 1.0813,
 'rho': 10.009,
 'son': 924.12,
 'cond': 1.9359,
 'pran': 0.65969,
 'gamma': 1.240134746}